In [2]:
#imports
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from nltk import word_tokenize
import spacy 
import en_core_web_sm


In [3]:
df = pd.read_csv('sentence_classifications_.csv')

In [4]:
df.head()

,Unnamed: 0,article_id,sentence_char_start,sentence_char_end,propaganda,propaganda_type,text,prop_start_char,prop_end_char,prop_txt_snippet,sent_#
0,0,701225819,0,59,non-propaganda,NaN,South Florida Muslim Leader Sofian Zakkout’s D...,NaN,NaN,NaN,1
1,1,701225819,60,207,propaganda,"Name_Calling,Labeling","David Duke, the white supremacist icon and for...",111.0,143.0,Grand Wizard of the Ku Klux Klan,2
2,2,701225819,207,382,propaganda,Loaded_Language,"However, one individual who represents the Mus...",305.0,313.0,enamored,3
3,3,701225819,382,525,non-propaganda,NaN,"Last month, once again, Zakkout chose to showc...",NaN,NaN,NaN,4
4,4,701225819,525,595,non-propaganda,NaN,The postings can be rivaled only by Zakkout’s ...,NaN,NaN,NaN,5


In [19]:
df_1 = df[df['text'].isna()==False]
df_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15220 entries, 0 to 15240
Data columns (total 11 columns):
Unnamed: 0             15220 non-null int64
article_id             15220 non-null int64
sentence_char_start    15220 non-null int64
sentence_char_end      15220 non-null int64
propaganda             15220 non-null object
propaganda_type        3840 non-null object
text                   15220 non-null object
prop_start_char        3840 non-null float64
prop_end_char          3840 non-null float64
prop_txt_snippet       3840 non-null object
sent_#                 15220 non-null int64
dtypes: float64(2), int64(5), object(4)
memory usage: 1.4+ MB


In [5]:
#LOOKING FOR DIFFERENCES IN PROP AND NON PROP FOR CERTAIN WORDS/TOPICS... MIGHT NEED TO CHANGE SOME LABELS

In [23]:
non_prop = df_1[(df_1['propaganda']=='non-propaganda')]
prop = df_1[(df_1['propaganda']=='propaganda')]
print(non_prop[non_prop['text'].str.contains('Arab ')].text)
print(prop[prop.text.str.contains('Arab ')].text)
print(len(non_prop[non_prop.text.str.contains('Arab ')]),len(prop[prop.text.str.contains('Arab ')]))


1616     They floated the idea later Monday of claiming...
9383     Unlike Washington’s Arab allies in the region,...
13995    In Tunisia, the release of WikiLeaks documents...
Name: text, dtype: object
3188     Together they appeared to signal a shift in Wh...
8457     The Nazis suggested Madagascar, Ecuador, or an...
8529             No one defends Islam like Arab Christians
13437    Second, in a reflexive response to 9/11, we in...
Name: text, dtype: object
3 4


## Feature Extraction From Sentences

### Sentence Sentiment Score

In [24]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

In [25]:
def sentiment_scores(sentence):
    snt = analyser.polarity_scores(sentence)
    return snt

In [26]:
def sentence_sentiment_score(new_col, transform_col, df):
    df[new_col] = df[transform_col].apply(lambda x: sentiment_scores(x)['compound'])
    return df

In [28]:
df_1 = sentence_sentiment_score('sentiment_score','text',df_1)

/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [29]:
df_1.head()

,Unnamed: 0,article_id,sentence_char_start,sentence_char_end,propaganda,propaganda_type,text,prop_start_char,prop_end_char,prop_txt_snippet,sent_#,sentiment_score
0,0,701225819,0,59,non-propaganda,NaN,South Florida Muslim Leader Sofian Zakkout’s D...,NaN,NaN,NaN,1,0.0000
1,1,701225819,60,207,propaganda,"Name_Calling,Labeling","David Duke, the white supremacist icon and for...",111.0,143.0,Grand Wizard of the Ku Klux Klan,2,0.5423
2,2,701225819,207,382,propaganda,Loaded_Language,"However, one individual who represents the Mus...",305.0,313.0,enamored,3,0.3612
3,3,701225819,382,525,non-propaganda,NaN,"Last month, once again, Zakkout chose to showc...",NaN,NaN,NaN,4,0.0000
4,4,701225819,525,595,non-propaganda,NaN,The postings can be rivaled only by Zakkout’s ...,NaN,NaN,NaN,5,0.0000


### Sentence Punctuation Count

In [30]:
import string  
punctuation = string.punctuation

In [31]:
punctuation = punctuation + '”'
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~”'

In [32]:
def count_punct(text):
    count = 0
    for char in text:
        if char in punctuation:
            count +=1
    return count

def count_punct_col(df,transform_col,new_col):
    df[new_col]=df[transform_col].apply(lambda x: count_punct(x))
    return df
    

In [33]:
test_string = 'He said: "HELLO!"... but that is it... no more'
count_punct(test_string)

10

In [35]:
df_1 = count_punct_col(df_1,'text','punct_count')

/Users/sashaepelbaum/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [36]:
df_1.head()

,Unnamed: 0,article_id,sentence_char_start,sentence_char_end,propaganda,propaganda_type,text,prop_start_char,prop_end_char,prop_txt_snippet,sent_#,sentiment_score,punct_count
0,0,701225819,0,59,non-propaganda,NaN,South Florida Muslim Leader Sofian Zakkout’s D...,NaN,NaN,NaN,1,0.0000,0
1,1,701225819,60,207,propaganda,"Name_Calling,Labeling","David Duke, the white supremacist icon and for...",111.0,143.0,Grand Wizard of the Ku Klux Klan,2,0.5423,4
2,2,701225819,207,382,propaganda,Loaded_Language,"However, one individual who represents the Mus...",305.0,313.0,enamored,3,0.3612,4
3,3,701225819,382,525,non-propaganda,NaN,"Last month, once again, Zakkout chose to showc...",NaN,NaN,NaN,4,0.0000,5
4,4,701225819,525,595,non-propaganda,NaN,The postings can be rivaled only by Zakkout’s ...,NaN,NaN,NaN,5,0.0000,1


### Sentence Word Count

### Sentence Avg. Word Length

### Sentence Vocabulary Complexity

In [ ]:
import random
import numpy
from matplotlib import pyplot

x = [random.gauss(3,1) for _ in range(400)]
y = [random.gauss(4,2) for _ in range(400)]

bins = numpy.linspace(-10, 10, 100)

pyplot.hist(x, bins, alpha=0.5, label='x')
pyplot.hist(y, bins, alpha=0.5, label='y')
pyplot.legend(loc='upper right')
pyplot.show()

In [159]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import word_tokenize, pos_tag
from collections import defaultdict
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

text = "guru99 is a totally new kind of learning experience."
tokens = word_tokenize(text)
lemma_function = WordNetLemmatizer()
for token, tag in pos_tag(tokens):
    lemma = lemma_function.lemmatize(token, tag_map[tag[0]])
    print(token, "=>", lemma)


[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sashaepelbaum/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


LookupError: 
**********************************************************************
  Resource [93maveraged_perceptron_tagger[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('averaged_perceptron_tagger')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtaggers/averaged_perceptron_tagger/averaged_perceptron_tagger.pickle[0m

  Searched in:
    - '/Users/sashaepelbaum/nltk_data'
    - '/Users/sashaepelbaum/anaconda3/nltk_data'
    - '/Users/sashaepelbaum/anaconda3/share/nltk_data'
    - '/Users/sashaepelbaum/anaconda3/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************
